In [1]:
import sys
sys.dont_write_bytecode = True
from collections import OrderedDict
import multiprocessing
import math
from breeder_blanket_model_maker import *
from breeder_blanket_model_maker.rewrite_cad_files import *
from breeder_blanket_model_maker.serpent_tools import *
from breeder_blanket_model_maker.make_serpent_input_file import *

from neutronics_material_maker.nmm import *
from neutronics_material_maker.examples import *


def define_blanket_geometry_parmeters(blanket_type,input_files,output_directory,poloidal_lithium_lead_in_mm = 35.4):

    poloidal_lithium_lead_in_mm = float(poloidal_lithium_lead_in_mm)
    smd = 274. * 1000000.  # units Pa

    cooling_plates_channel_poloidal_mm = 3.0
    cooling_plates_channel_radial_mm = 3.0
    cooling_plates_channel_toroidal_mm = 6.0
    first_wall_channel_radial_mm = 10.0
    first_wall_channel_poloidal_mm = 15.0
    first_wall_channel_pitch_mm = 5.2

    pressure_coolant = 10. * 1000000.  # in units Pa

    first_walls_thickness = (first_wall_channel_radial_mm/1000) + math.sqrt((pressure_coolant * (poloidal_lithium_lead_in_mm/1000) * (poloidal_lithium_lead_in_mm/1000) ) / (4. * smd))  # units in m
    first_walls_thickness_mm = first_walls_thickness *1000

    poloidal_cooling_plate_mm = (pressure_coolant * (poloidal_lithium_lead_in_mm)) / (1.1 * smd)

    poloidal_cooling_plate_mm = max(poloidal_cooling_plate_mm, 0.002*1000) + cooling_plates_channel_poloidal_mm

    cooling_channel_offset_from_first_wall = (first_walls_thickness_mm - first_wall_channel_radial_mm) / 2.0

    if cooling_channel_offset_from_first_wall < 1.0:
        raise ValueError('first wall is too thin at ',cooling_channel_offset_from_first_wall, 'mm')

    poloidal_segmentations_ordered_dict=OrderedDict()
    poloidal_segmentations_ordered_dict['lithium_lead']=poloidal_lithium_lead_in_mm
    poloidal_segmentations_ordered_dict['cooling_plate_homogenised']=poloidal_cooling_plate_mm

    back_plates_thicknesses_ordered_dict=OrderedDict()  #these are special ordereddict types required in python 2.7 but in python 3.6 onwards dictionaries are ordered by default, sadly freecad is not yet avaialbe in python 3
    back_plates_thicknesses_ordered_dict['back_plate_1']=30
    back_plates_thicknesses_ordered_dict['back_helium']=50
    back_plates_thicknesses_ordered_dict['back_plate_2']=30
    back_plates_thicknesses_ordered_dict['back_lithium_lead']=25
    back_plates_thicknesses_ordered_dict['back_plate_3']=10

    first_wall_channel_poloidal_segmentations_dict=OrderedDict()
    first_wall_channel_poloidal_segmentations_dict['first_wall_material']=first_wall_channel_poloidal_mm
    first_wall_channel_poloidal_segmentations_dict['first_wall_coolant']=first_wall_channel_pitch_mm


    list_of_compressed_arguments=[]
    for module in input_files:

        blanket_geometry_parameters =  {

            'blanket_type' : blanket_type,
            'plasma_filename' :'/home/jshim/Eurofusion_baseline_2016/envelopes/plasma.step',
            'envelope_filename' : module,
            'output_folder' : output_directory,
            'output_files':['step','stl'],
            'first_wall_poloidal_fillet_radius' : 50,
            'armour_thickness' : 2,
            'first_wall_thickness' : first_walls_thickness_mm,
            'end_cap_thickness' : 25,
            'back_plates_thicknesses' : back_plates_thicknesses_ordered_dict,
            'poloidal_segmentations' : poloidal_segmentations_ordered_dict,

        }

        if module == input_files[13]:
            blanket_geometry_parameters.update({
                #The following three arguments are optional, if included the model will include cooling channels on the first wall
                'cooling_channel_offset_from_first_wall': cooling_channel_offset_from_first_wall,
                'first_wall_channel_radial_mm': first_wall_channel_radial_mm,
                'first_wall_channel_poloidal_segmentations': first_wall_channel_poloidal_segmentations_dict,  # 13.5,4.5

                # #The following two arguments are optional, if included the model will include a slice of blanket and cooling channels in the cooling plate
                'cooling_plates_channel_poloidal_mm':cooling_plates_channel_poloidal_mm,
                'cooling_plates_channel_radial_mm':cooling_plates_channel_radial_mm,

                'output_files':['step','stl','umesh'],
                })



        list_of_compressed_arguments.append(blanket_geometry_parameters)

    return list_of_compressed_arguments

def define_neutronics_materials(enrichment_fraction):

    mat_lithium_lead =Compound('Pb84.2Li15.8',
                              density_atoms_per_barn_per_cm=3.2720171E-2,
                              enriched_isotopes=[Isotope('Li',6,abundance=enrichment_fraction),
                                                 Isotope('Li',7,abundance=1.0-enrichment_fraction)])

    material_dictionary = {  'armour': mat_Tungsten,
                             'lithium_lead': mat_lithium_lead,
                             'back_plate_1': mat_Eurofer,
                             'back_plate_2': mat_Eurofer,
                             'back_plate_3': mat_Eurofer,
                             'back_lithium_lead': mat_lithium_lead,
                             'back_helium': mat_He_coolant_back_plate,
                             'cooling_plate_homogenised': mat_cooling_plates_homogenised,
                             'end_caps_homogenised': mat_end_caps_homogenised,
                             'first_wall_homogenised': mat_first_wall_homogenised,
                             'plasma': mat_DT_plasma,
                             'central_solenoid': mat_central_solenoid_m25,
                             'divertor_1st_layer': mat_divertor_layer_1_m15, 
                             'divertor_2nd_layer': mat_divertor_layer_2_m74, 
                             'divertor_3rd_layer': mat_divertor_layer_3_m15,
                             'divertor_4th_layer': mat_divertor_layer_4_m75,
                             'manifolder': mat_VV_Body_m60 ,
                             'ports': mat_TF_Casing_m50,
                             #'shell': , # outer shell ignored
                             'shield': mat_TF_Casing_m50,
                             'tf_case': mat_TF_Casing_m50,
                             'tf_coils': mat_TF_Magnet_m25,
                             'vacuum_1st_layer': mat_VV_Shell_m50,
                             'vacuum_2nd_layer': mat_VV_Body_m60,
                             'vacuum_3rd_layer': mat_VV_Shell_m50,
                             'vaccum_vessel_shield': mat_ShieldPort_m60,
                             'blanket_support':mat_Eurofer,

                             'first_wall_coolant':mat_He_in_first_walls,
                             'first_wall_material':mat_He_in_first_walls,

                             'slice_lithium_lead':mat_lithium_lead,
                             'slice_armour':mat_Tungsten,
                             'slice_first_wall_material':mat_Eurofer,
                             'slice_first_wall_homogenised':mat_first_wall_homogenised, #ignore
                             'slice_first_wall_coolant':mat_He_in_first_walls,
                             'slice_cooling_plate_material':mat_cooling_plates_homogenised,
                             'slice_cooling_plate_coolant':mat_He_in_coolant_plates,
                             'slice_back_plate_1':mat_Eurofer,
                             'slice_back_helium':mat_He_coolant_back_plate,
                             'slice_back_plate_2':mat_Eurofer,
                             'slice_back_lithium_lead':mat_lithium_lead,
                             'slice_back_plate_3':mat_Eurofer,
                          }
    return material_dictionary

def define_neutronics_model_parmeters(list_detailed_modules_parts,material_dictionary,output_directory,nps=1e7,**kwargs):

    neutronics_parameters= { 'output_folder':output_directory,
                             'parts':list_detailed_modules_parts,
                             'include_umesh':True,
                             'output_folder_stl':os.path.join(output_directory,'stl'),
                             'material_dictionary':material_dictionary,
                             'plot_serpent_geometry':True,
                             'nps':nps,
                             'tallies':[{'name': 'tbr',
                                         'bodies': ['lithium_lead'],
                                         'mt_number':-55,
                                         'particle_type':'n'},
                                        {'name': 'neutron_heating',
                                         'bodies': ['armour', 'lithium_lead', 'back_plate_1','back_plate_2','back_plate_3','back_lithium_lead','back_helium','cooling_plate_homogenised','end_caps_homogenised','first_wall_homogenised',],
                                         'mt_number':-4,
                                         'particle_type':'n'},
                                        {'name': 'photon_heating',
                                         'bodies': ['armour', 'lithium_lead', 'back_plate_1','back_plate_2','back_plate_3','back_lithium_lead','back_helium','cooling_plate_homogenised','end_caps_homogenised','first_wall_homogenised',],
                                         'mt_number':-26,
                                         'particle_type':'p'},
                                       ]
                             }

    return neutronics_parameters


output_directory='/home/jshim/detailed_HCLL'
list_of_geometry_parameters = define_blanket_geometry_parmeters(blanket_type ='HCLL',
                                                                input_files= ['/home/jshim/Eurofusion_baseline_2016/envelopes/mod' + str(x) + '.step' for x in range(1,27)],#27,
                                                                output_directory = output_directory,
                                                                poloidal_lithium_lead_in_mm =80.0)

list_of_detailed_modules_parts = detailed_module(list_of_geometry_parameters)


example materials imported
Creating detailed HCLL
'read_in_envelope_file'  5.31 ms
'read_in_envelope_file'  10.32 ms
'read_in_envelope_file'  9.93 ms
'read_in_envelope_file'  8.82 ms
'read_in_envelope_file'  9.68 ms
'read_in_envelope_file'  10.58 ms
'read_in_envelope_file'  9.67 ms
'find_envelope_back_face'  523.73 ms
'find_envelope_front_face'  10.16 ms
'find_front_face_midpoint'  0.24 ms
'envelope_front_face_id'  10.46 ms
'find_envelope_back_face'  689.85 ms
'find_envelope_front_face'  12.63 ms
'find_front_face_midpoint'  0.25 ms
'envelope_front_face_id'  10.65 ms
'find_envelope_back_face_id'  546.14 ms
'find_front_face_polodial_edges_to_fillet'  0.07 ms
'find_front_face_torodial_edges_to_fillet'  0.23 ms
'filleted_envelope'  13.40 ms
'find_envelope_back_face'  1473.76 ms
'find_envelope_front_face'  9.98 ms
'find_front_face_midpoint'  0.25 ms
'envelope_front_face_id'  9.73 ms
'find_envelope_back_face_id'  859.07 ms
'find_front_face_polodial_edges_to_fillet'  0.11 ms
'find_front_face_

'find_envelope_back_face'  3195.28 ms
'find_envelope_front_face'  11.50 ms
'find_front_face_midpoint'  0.33 ms
'exstrude_and_cut_solids'  769.67 ms
'chop_of_end_caps'  158.22 ms
'chop_up_poloidally'  1628.64 ms
'make_cylinder_slice'  0.16 ms
'find_envelope_back_face'  2695.50 ms
'find_envelope_front_face'  16.01 ms
'find_envelope_back_face'  3139.73 ms
'find_envelope_front_face'  11.15 ms
'envelope_front_face_id'  16.14 ms
'find_front_face_midpoint'  0.18 ms
'find_end_cap_faces'  0.12 ms
'make_cylinder_slice'  0.24 ms
'make_cylinder_slice'  0.23 ms
'make_cylinder_slice'  0.23 ms
'make_cylinder_slice'  0.29 ms
'chop_of_end_caps'  188.27 ms
'make_cylinder_slice'  0.45 ms
'chop_off_first_wall_armour'  274.59 ms
'make_cylinder_slice'  2.30 ms
'make_cylinder_slice'  0.39 ms
'make_cylinder_slice'  0.26 ms
'make_cylinder_slice'  0.25 ms
'save_components_as_step'  730.52 ms
'find_envelope_back_face'  3218.01 ms
'find_envelope_front_face'  11.09 ms
'find_front_face_midpoint'  0.21 ms
'chop_of_e

'chop_off_first_wall_armour'  370.33 ms
'find_envelope_back_face'  1634.93 ms
'find_envelope_front_face'  12.32 ms
'find_front_face_midpoint'  0.23 ms
'envelope_front_face_id'  17.01 ms
'exstrude_and_cut_solids'  2886.32 ms
'chop_up_poloidally'  5812.30 ms
'make_cylinder_slice'  0.18 ms
'find_envelope_back_face'  1646.46 ms
'find_envelope_front_face'  7.86 ms
'find_front_face_midpoint'  0.16 ms
'envelope_front_face_id'  11.72 ms
'save_components_as_stl'  894.91 ms
'find_envelope_back_face'  2069.64 ms
'find_envelope_front_face'  14.57 ms
'find_front_face_midpoint'  0.25 ms
'chop_of_end_caps'  167.94 ms
'read_in_envelope_file'  9.83 ms
'find_envelope_back_face'  2109.64 ms
'find_envelope_front_face'  12.75 ms
'find_end_cap_faces'  0.12 ms
'make_cylinder_slice'  2.11 ms
'make_cylinder_slice'  0.30 ms
'make_cylinder_slice'  0.32 ms
'find_envelope_back_face_id'  1345.20 ms
'find_front_face_polodial_edges_to_fillet'  0.08 ms
'find_front_face_torodial_edges_to_fillet'  0.06 ms
'filleted_enve

'chop_off_back_plates'  724.20 ms
'convert_distances_into_cumlative_distances'  0.03 ms
'exstrude_and_cut_solids'  1930.38 ms
'read_in_envelope_file'  7.92 ms
'find_envelope_back_face'  892.08 ms
'find_envelope_back_face_id'  444.19 ms
'find_front_face_polodial_edges_to_fillet'  0.07 ms
'find_front_face_torodial_edges_to_fillet'  0.07 ms
'find_envelope_front_face'  11.96 ms
'filleted_envelope'  12.87 ms
'chop_off_first_wall'  89.66 ms
'find_envelope_back_face'  280.70 ms
'find_envelope_front_face'  6.93 ms
'find_front_face_midpoint'  0.23 ms
'envelope_front_face_id'  7.57 ms
'find_envelope_back_face_id'  256.01 ms
'find_front_face_polodial_edges_to_fillet'  0.08 ms
'find_front_face_torodial_edges_to_fillet'  0.06 ms
'filleted_envelope'  10.38 ms
'make_cylinder_slice'  2.43 ms
'make_cylinder_slice'  0.37 ms
'make_cylinder_slice'  0.23 ms
'find_envelope_back_face'  727.08 ms
'make_cylinder_slice'  0.28 ms
'find_envelope_front_face'  19.19 ms
'envelope_front_face_id'  21.04 ms
'find_end_c

'find_front_face_torodial_edges_to_fillet'  0.08 ms
'chop_off_back_plates'  393.23 ms
'convert_distances_into_cumlative_distances'  0.03 ms
'filleted_envelope'  12.31 ms
'find_envelope_back_face'  738.89 ms
'find_envelope_front_face'  12.57 ms
'envelope_front_face_id'  12.58 ms
'find_end_cap_faces'  0.09 ms
'find_envelope_back_face'  646.63 ms
'find_envelope_front_face'  9.89 ms
'find_front_face_midpoint'  0.19 ms
'envelope_front_face_id'  10.61 ms
'chop_off_first_wall_armour'  206.80 ms
'exstrude_and_cut_solids'  2057.67 ms
'find_envelope_back_face'  884.29 ms
'find_envelope_front_face'  15.90 ms
'envelope_front_face_id'  18.24 ms
'find_end_cap_faces'  0.14 ms
'find_envelope_back_face'  1093.73 ms
'find_envelope_back_face_id'  649.99 ms
'find_envelope_front_face'  11.65 ms
'find_front_face_polodial_edges_to_fillet'  0.08 ms
'find_front_face_torodial_edges_to_fillet'  0.09 ms
'filleted_envelope'  12.72 ms
'find_envelope_back_face'  631.67 ms
'find_envelope_front_face'  12.58 ms
'find_e

('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
('cooling channel formation cut_solid.Volume', 0.0)
'add_cooling_pipes_to_div'  3638.57 ms
'make_cylinder_slice'  0.20 ms
'make_cylinder_slice'  0.19 ms
'make_cylinder_slice'  0.14 ms
'make_cylinder_slice'  0.20 ms
'make_cylinder_slice'  0.38 ms
'make_cylinder_slice'  0.66 ms
'make_cylinder_slice'  7.40 ms
'make_cylinder_slice'  0.24 ms
'make_cylinder_slice'  0.19 ms
'make_cylinder_slice'  0.21 ms
'make_cylinder_slice'  0.24 ms
'make_cylinder_slice'  0.25 ms
'save_components_

In [2]:
list_of_detailed_modules_parts


[defaultdict(dict,
             {'armour': {'part': [<Shape object at 0x7f628c7801e0>],
               'step_filename': '/home/jshim/detailed_HCLL/step/armour_mod1.step',
               'stl_filename': ['/home/jshim/detailed_HCLL/stl/armour_mod1_1.stl']},
              'back_helium': {'part': [<Shape object at 0x7f628c83b560>],
               'step_filename': '/home/jshim/detailed_HCLL/step/back_helium_mod1.step',
               'stl_filename': ['/home/jshim/detailed_HCLL/stl/back_helium_mod1_1.stl']},
              'back_lithium_lead': {'part': [<Shape object at 0x7f628c77a6b0>],
               'step_filename': '/home/jshim/detailed_HCLL/step/back_lithium_lead_mod1.step',
               'stl_filename': ['/home/jshim/detailed_HCLL/stl/back_lithium_lead_mod1_1.stl']},
              'back_plate_1': {'part': [<Shape object at 0x7f628c778760>],
               'step_filename': '/home/jshim/detailed_HCLL/step/back_plate_1_mod1.step',
               'stl_filename': ['/home/jshim/detailed_HCLL

In [3]:
extra_parts = read_in_step_files_and_save_as_seperate_stl_files(read_folder='/home/jshim/Eurofusion_baseline_2016/reactor_step_files',
                                                                write_folder=os.path.join(output_directory,'stl'),
                                                                ignore_files=['shell.stp'])

('error making folder', '/home/jshim/detailed_HCLL/stl')
('read_folder', '/home/jshim/Eurofusion_baseline_2016/reactor_step_files')
['shield.stp', 'vacuum_1st_layer.stp', 'divertor_4th_layer.stp', 'divertor_3rd_layer.stp', 'divertor_2nd_layer.stp', 'vaccum_vessel_shield.stp', 'plasma.stp', 'divertor_1st_layer.stp', 'blanket_support.stp', 'central_solenoid.stp', 'vacuum_2nd_layer.stp', 'manifolder.stp', 'ports.stp', 'shell.stp', 'vacuum_3rd_layer.stp', 'tf_coils.stp', 'tf_case.stp']
shield.stp
vacuum_1st_layer.stp
divertor_4th_layer.stp
divertor_3rd_layer.stp
divertor_2nd_layer.stp
vaccum_vessel_shield.stp
plasma.stp
divertor_1st_layer.stp
blanket_support.stp
central_solenoid.stp
vacuum_2nd_layer.stp
manifolder.stp
ports.stp
vacuum_3rd_layer.stp
tf_coils.stp
tf_case.stp


In [4]:
extra_parts

defaultdict(dict,
            {'blanket_support': {'part': <Shape object at 0x5ae7fb0>,
              'stl_filename': ['/home/jshim/detailed_HCLL/stl/blanket_support_1.stl',
               '/home/jshim/detailed_HCLL/stl/blanket_support_2.stl',
               '/home/jshim/detailed_HCLL/stl/blanket_support_3.stl',
               '/home/jshim/detailed_HCLL/stl/blanket_support_4.stl',
               '/home/jshim/detailed_HCLL/stl/blanket_support_5.stl',
               '/home/jshim/detailed_HCLL/stl/blanket_support_6.stl',
               '/home/jshim/detailed_HCLL/stl/blanket_support_7.stl',
               '/home/jshim/detailed_HCLL/stl/blanket_support_8.stl',
               '/home/jshim/detailed_HCLL/stl/blanket_support_9.stl',
               '/home/jshim/detailed_HCLL/stl/blanket_support_10.stl',
               '/home/jshim/detailed_HCLL/stl/blanket_support_11.stl',
               '/home/jshim/detailed_HCLL/stl/blanket_support_12.stl',
               '/home/jshim/detailed_HCLL/stl/blanket

In [5]:
list_of_detailed_modules_parts.append(extra_parts)

In [7]:
for item in list_of_detailed_modules_parts:
    print(item.keys())

['cooling_plate_homogenised', 'back_lithium_lead', 'back_plate_1', 'armour', 'back_plate_2', 'end_caps_homogenised', 'first_wall_homogenised', 'back_helium', 'back_plate_3', 'lithium_lead']
['cooling_plate_homogenised', 'back_lithium_lead', 'back_plate_1', 'armour', 'back_plate_2', 'end_caps_homogenised', 'first_wall_homogenised', 'back_helium', 'back_plate_3', 'lithium_lead']
['cooling_plate_homogenised', 'back_lithium_lead', 'back_plate_1', 'armour', 'back_plate_2', 'end_caps_homogenised', 'first_wall_homogenised', 'back_helium', 'back_plate_3', 'lithium_lead']
['cooling_plate_homogenised', 'back_lithium_lead', 'back_plate_1', 'armour', 'back_plate_2', 'end_caps_homogenised', 'first_wall_homogenised', 'back_helium', 'back_plate_3', 'lithium_lead']
['cooling_plate_homogenised', 'back_lithium_lead', 'back_plate_1', 'armour', 'back_plate_2', 'end_caps_homogenised', 'first_wall_homogenised', 'back_helium', 'back_plate_3', 'lithium_lead']
['cooling_plate_homogenised', 'back_lithium_lead',

In [9]:
material_dictionary=define_neutronics_materials(enrichment_fraction=0.8)

neutronics_parameters = define_neutronics_model_parmeters(list_detailed_modules_parts=list_of_detailed_modules_parts,
                                                          material_dictionary=material_dictionary,
                                                          output_directory=output_directory,
                                                          nps=100000)

In [18]:
def find_components(list_of_detailed_modules_components):
    #print('parts = ',list_of_detailed_modules_components)
    #print(type)
    dictionary_of_components=collections.defaultdict(list)

    for list_entry in list_of_detailed_modules_components:
            for key,value in list_entry.iteritems():
                dictionary_of_components[key].append(value)

    return dictionary_of_components


In [19]:
find_components(list_of_detailed_modules_parts)

defaultdict(list,
            {'armour': [{'part': [<Shape object at 0x7f628c7801e0>],
               'step_filename': '/home/jshim/detailed_HCLL/step/armour_mod1.step',
               'stl_filename': ['/home/jshim/detailed_HCLL/stl/armour_mod1_1.stl']},
              {'part': [<Shape object at 0x7f628c144e20>],
               'step_filename': '/home/jshim/detailed_HCLL/step/armour_mod2.step',
               'stl_filename': ['/home/jshim/detailed_HCLL/stl/armour_mod2_1.stl']},
              {'part': [<Shape object at 0x7f628c36d450>],
               'step_filename': '/home/jshim/detailed_HCLL/step/armour_mod3.step',
               'stl_filename': ['/home/jshim/detailed_HCLL/stl/armour_mod3_1.stl']},
              {'part': [<Shape object at 0x7f628c09fdc0>],
               'step_filename': '/home/jshim/detailed_HCLL/step/armour_mod4.step',
               'stl_filename': ['/home/jshim/detailed_HCLL/stl/armour_mod4_1.stl']},
              {'part': [<Shape object at 0x7f628c54c270>],
     